<a href="https://colab.research.google.com/github/IsaacFigNewton/Taxonomic-Span-Categorization/blob/main/Test_Taxonomic_NER.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install dependencies

In [ ]:
!pip install git+https://github.com/IsaacFigNewton/Taxonomic-Span-Categorization.git

# Config

In [ ]:
import json
import spacy
from spacy import displacy
from tax_span_cat.SpanCategorizer import SpanCategorizer

In [ ]:
nlp = spacy.load("en_core_web_sm")
ner = SpanCategorizer(taxonomic_features=["wordnet_synsets"])

# Test NER functionality

In [ ]:
doc = nlp("In 1999, Tim Berners-Lee was one of the first to introduce the idea of the Semantic Web.")
ner_doc = ner(doc)
ner_doc.ents

In [ ]:
# TODO: find out why this breaks
json.dumps(ner_doc.spans, indent=4, default=str)

In [ ]:
displacy.render(ner_doc, style="ent", jupyter=True)